In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Introducing Z3

In this notebook we are going to solve the following exercise using the constraint solver `Z3`.

<p>
<center style="color:blue; background-color:#FED8B1; font-family: Philosopher; font-size:20px; padding: 8px">
    I have as many brothers as I have sisters, but my sister has twice as many brothers as she has sisters.
    How many children does my father have?
</center>
</p>

First, we have to install `Z3`.  This can be achieved via the following commands:

In [2]:
!pip install z3
!pip install z3-solver

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for z3: filename=z3-0.2.0-py3-none-any.whl size=26630 sha256=0eff15bf74cc8f5d1118717a0f29768dd4ebaae08895c9670159b0c659c8fc23
  Stored in directory: /Users/stroetmann/Library/Caches/pip/wheels/5a/b2/60/55b07a5084cad7ab411e395fb1440a2b1a19598bff535a3955
Successfully built z3
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 10.3 MB/s eta 0:00:0000:0100:01


Next, we import the Python API of `Z3`. 

In [3]:
import z3

In `Z3`, every variable has to be declared.  In this case, all our variables are integer valued.  The function `Int(v)` declares a variable with the name `v`.
The object `z3.Int('boys')` is the `Z3` representation of a constraint variable.  We assign this object to the Python variable `boys`. 

In [4]:
boys  = z3.Int('boys')
girls = z3.Int('girls')

Next, we create a *solver* object.  This is a constraint solver capable of solving constraint satisfaction problems.

In [5]:
S = z3.Solver()

We can add constraints to this solver object via the method `add`.
We will use two variables:
 - `boys` is the number of my father's sons,
 - `girls` is the number of my father's daughters.
 
Note that the number of my brothers is `boys - 1` as I am not my own brother.  Similarly, the number of my sister's sisters is `sisters - 1`.
So the statement that I have as many brothers as sisters, but my sister has twice as many brothers as she has sisters is formalized as follows:

In [6]:
S.add(boys - 1 == girls)
S.add(2 * (girls - 1) == boys)

The method `check` examines whether the given set of constraints is satisfiable.
In general, it can return one of the following results:
- `sat`   is returned if the problem is solvable, (`sat` is short for *satisfiable*)
- `unsat` is returned if the problem is unsatisfiable,
- `unknown` is returned if the constraint solver is not powerful enough to solve the given problem.

  In contrast to *ChatGPT*, `Z3` does not start to halucinate if it is unable to solve a given problem, but instead clearly states that it was unable to solve the problem.

In [7]:
if str(S.check()) == 'sat':
    print('satisfiable')

satisfiable


To extract the solution of the given problem we use the method `model`.

In [8]:
Solution = S.model()
Solution

[boys = 4, girls = 3]

In order to extract the number of boys and girls we can use the dictionary syntax
`Solution[boys]` and `Solution[girls]`.  In order to convert these results into 
Python integers, we use the mehod `as_long`.

In [9]:
b = Solution[boys ].as_long()
g = Solution[girls].as_long()

In [10]:
print(f'My father has {b + g} children: {b} boys and {g} girls.')
print(f'I have {b} - 1 = {b-1} brother and {g} sisters.')
print(f'My sister has {g} - 1 = {g-1} sisters and {b} = 2 * {g-1}  brothers.')

My father has 7 children: 4 boys and 3 girls.
I have 4 - 1 = 3 brother and 3 sisters.
My sister has 3 - 1 = 2 sisters and 4 = 2 * 2  brothers.
